In [382]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
import json

In [383]:
data = pd.read_csv("./data/train.csv").drop('Ticket',axis=1).drop('Cabin',axis=1).drop('Name',axis=1)
data["Sex"][data["Sex"] == "male"] = 0
data["Sex"][data["Sex"] == "female"] = 1
print(data.shape)
data.head()

(891, 9)


/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,S
1,2,1,1,1,38.0,1,0,71.2833,C
2,3,1,3,1,26.0,0,0,7.9250,S
3,4,1,1,1,35.0,1,0,53.1000,S
4,5,0,3,0,35.0,0,0,8.0500,S


In [384]:
data["Age"] = data["Age"].fillna(data["Age"].median())
data["Embarked"] = data["Embarked"].fillna("S")


data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [385]:
dummy_columns = ['Embarked']

dummy = pd.get_dummies(data[dummy_columns])
data = pd.concat([data, dummy], axis=1).drop('Embarked',axis=1).astype(float)

data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0
1,2.0,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0
2,3.0,1.0,3.0,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0
3,4.0,1.0,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0
4,5.0,0.0,3.0,0.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0


In [386]:
data.shape

(891, 11)

In [387]:
data_feature = data.loc[:, 'Pclass':'Embarked_S'].values
data_target = data.loc[:, 'Survived'].values


data_feature

array([[  3.,   0.,  22., ...,   0.,   0.,   1.],
       [  1.,   1.,  38., ...,   1.,   0.,   0.],
       [  3.,   1.,  26., ...,   0.,   0.,   1.],
       ..., 
       [  3.,   1.,  28., ...,   0.,   0.,   1.],
       [  1.,   0.,  26., ...,   1.,   0.,   0.],
       [  3.,   0.,  32., ...,   0.,   1.,   0.]])

In [388]:
data_train_feature = data_feature[:700]
data_train_target = data_target[:700]

data_test_feature = data_feature[700:]
data_test_target = data_target[700:]

In [389]:
INPUT_COLUMNS = len(data_feature[0])
LAYER = 20
OUTPUT_SIZE=2

In [390]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(INPUT_COLUMNS,INPUT_COLUMNS)
        self.fc2 = nn.Linear(INPUT_COLUMNS,int(LAYER/2))
        self.fc3 = nn.Linear(int(LAYER/2),LAYER)
        self.fc4 = nn.Linear(LAYER,OUTPUT_SIZE)
        #self.outact = nn.Sigmoid()
        self.outact = nn.Softmax()
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return self.outact(x)

In [391]:
def TrainingNN(model,train_loader):

    #criterion = nn.BCELoss()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=0.01)

    for epoch in range(300):
        total_loss = 0
    
        for train_x, train_y in train_loader:
            train_x, train_y = Variable(train_x), Variable(train_y)
            optimizer.zero_grad()
            output = model(train_x)
            loss = criterion(output, train_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.data[0]
    
        if (epoch+1) % 50 == 0:
            print(epoch+1, total_loss)

In [392]:
#ロジスティック回帰

from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(data_feature, data_target, random_state=0)
logreg = LogisticRegression().fit(X_train, y_train)

score = logreg.score(X_test, y_test)
print('Test set score: {}'.format(score))

Test set score: 0.7982062780269058


In [393]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((data_train_target.shape[0],))
    oof_test = np.zeros((data_test_target.shape[0],))
    oof_test_skf = np.empty((3, data_test_target.shape[0]))

    for i, (train_index, test_index) in enumerate(StratifiedKFold(n_splits=3).split(data_train_feature, data_train_target)): # NFOLDS回まわる
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.fit(x_tr, y_tr)

        #oof_train[test_index] = clf.predict(x_te)
        #oof_test_skf[i, :] = clf.predict(x_test)
        oof_train[test_index] = clf.predict_proba(x_te)[:,1]
        oof_test_skf[i, :] = clf.predict_proba(x_test)[:,1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


In [394]:
lg.predict_proba(data_train_feature)[:,1]
#X_test = torch.from_numpy(data_train_feature).float()
#test = Variable(X_test)
#model(test).data[:,1]

array([ 0.10276096,  0.8333127 ,  0.65042907,  0.82575704,  0.09519348,
        0.21246011,  0.21329924,  0.10904125,  0.67427384,  0.82986627,
        0.74946709,  0.74600313,  0.14851213,  0.08660346,  0.73597886,
        0.60073189,  0.15051581,  0.22140399,  0.53039173,  0.66128305,
        0.18747255,  0.18851326,  0.84510907,  0.38632127,  0.5663393 ,
        0.56163519,  0.13003129,  0.35206084,  0.77874057,  0.1175091 ,
        0.31727062,  0.89108401,  0.77869453,  0.07288125,  0.35413728,
        0.22235069,  0.13003227,  0.14430091,  0.55187865,  0.68945434,
        0.45036256,  0.73319486,  0.13018828,  0.89214966,  0.82658808,
        0.11754217,  0.16149534,  0.77869453,  0.07083826,  0.6363127 ,
        0.07029762,  0.14423709,  0.77727513,  0.72188543,  0.18677113,
        0.38632127,  0.82388914,  0.12813802,  0.8698866 ,  0.04803055,
        0.15466428,  0.86552076,  0.21618592,  0.11036694,  0.41048747,
        0.10258909,  0.78131175,  0.1528533 ,  0.41380893,  0.06

In [395]:
def get_oof_NN(x_train, y_train, x_test):
    
    clf = Net()
    oof_train = np.zeros((data_train_target.shape[0],))
    oof_test = np.zeros((data_test_target.shape[0],))
    oof_test_skf = np.empty((3, data_test_target.shape[0]))

    for i, (train_index, test_index) in enumerate(StratifiedKFold(n_splits=3).split(data_train_feature, data_train_target)): # NFOLDS回まわる
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        
        train_X = torch.from_numpy(x_tr).float()
        train_Y = torch.from_numpy(y_tr).long()
        test_X = torch.from_numpy(x_te).float()

        train = TensorDataset(train_X, train_Y)
        train_loader = DataLoader(train, batch_size=16, shuffle=True)
        TrainingNN(clf,train_loader)
        
        test_x = Variable(test_X)
        X_test = torch.from_numpy(x_test).float()
        test = Variable(X_test)
        
        oof_train[test_index] = clf(test_x).data[:,1]
        oof_test_skf[i, :] = clf(test).data[:,1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)



In [396]:
print(type(model))
lg = LogisticRegression()
rf = RandomForestClassifier()
svc = svm.SVC(probability=True)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1)
model=Net()
lg_oof_train,lg_oof_test = get_oof(lg,data_train_feature,data_train_target,data_test_feature)
rf_oof_train, rf_oof_test = get_oof(rf,data_train_feature, data_train_target, data_test_feature)
gb_oof_train,gb_oof_test = get_oof(gb,data_train_feature,data_train_target,data_test_feature)
svc_oof_train,svc_oof_test = get_oof(svc,data_train_feature,data_train_target,data_test_feature)
nn_oof_train,nn_oof_test = get_oof_NN(data_train_feature,data_train_target,data_test_feature)



<class '__main__.Net'>


/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
/Users/GenseiYoshimura/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


50 tensor(15.2682)
100 tensor(15.3297)
150 tensor(15.0409)
200 tensor(15.0302)
250 tensor(14.3295)
300 tensor(15.5608)
50 tensor(15.6298)
100 tensor(15.8657)
150 tensor(15.2892)
200 tensor(14.4705)
250 tensor(15.3285)
300 tensor(15.1065)
50 tensor(14.6341)
100 tensor(17.0264)
150 tensor(14.6669)
200 tensor(14.7940)
250 tensor(15.0646)
300 tensor(14.7937)


In [397]:
svc_oof_test

array([[ 0.47222309],
       [ 0.6325324 ],
       [ 0.39704932],
       [ 0.23704306],
       [ 0.22025829],
       [ 0.33268469],
       [ 0.28993794],
       [ 0.32075404],
       [ 0.42957726],
       [ 0.25366277],
       [ 0.55000563],
       [ 0.32593394],
       [ 0.36013542],
       [ 0.22407097],
       [ 0.24628375],
       [ 0.22880918],
       [ 0.50324363],
       [ 0.45180691],
       [ 0.29971956],
       [ 0.22873317],
       [ 0.48787434],
       [ 0.24592412],
       [ 0.53550479],
       [ 0.2982439 ],
       [ 0.65163922],
       [ 0.22310582],
       [ 0.39835843],
       [ 0.54814637],
       [ 0.42913104],
       [ 0.36490293],
       [ 0.42071846],
       [ 0.59071956],
       [ 0.2310364 ],
       [ 0.26050227],
       [ 0.26050227],
       [ 0.25026662],
       [ 0.43510813],
       [ 0.71684662],
       [ 0.19516223],
       [ 0.19516223],
       [ 0.59771714],
       [ 0.59765133],
       [ 0.57518526],
       [ 0.37928585],
       [ 0.22270253],
       [ 0

In [398]:
len(oof_test)

191

In [399]:
oof_test_skf

array([[ -0.00000000e+000,  -0.00000000e+000,   8.48798317e-314, ...,
          6.93128705e-310,   6.93140108e-310,   2.19841359e-314],
       [  4.66839074e-313,   2.75859453e-313,   9.88131292e-324, ...,
          6.93140108e-310,   6.93140108e-310,   5.51718906e-313],
       [  5.54252145e-313,   5.00659867e-308,   1.03977794e-312, ...,
          1.35807731e-312,   1.48219694e-323,   6.93140108e-310],
       ..., 
       [  6.93140108e-310,   6.93140108e-310,   6.93140108e-310, ...,
          0.00000000e+000,   1.28457068e-322,   6.93140108e-310],
       [  6.93140108e-310,   4.94065646e-324,   8.48798319e-314, ...,
          0.00000000e+000,   4.03179328e-313,   0.00000000e+000],
       [  4.94065646e-323,   3.39519454e-313,   0.00000000e+000, ...,
          0.00000000e+000,   1.28457068e-322,   6.93140108e-310]])

In [400]:
oof_train[0]
print(len(oof_train))
print(len(oof_train))

700
700


In [401]:
base_predictions_train = pd.DataFrame( {'LogicticReg': lg_oof_train.ravel(),
      'RandomForest': rf_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel(),
      'SVC': svc_oof_train.ravel(),
      'NN': nn_oof_train.ravel()
    })
print('base_predictions_train.shape : ', base_predictions_train.shape)
base_predictions_train.head(5)

base_predictions_train.shape :  (700, 5)


,GradientBoost,LogicticReg,NN,RandomForest,SVC
0,0.094545,0.108456,5.759919e-37,0.3,0.253598
1,0.900090,0.929258,1.000000e+00,0.9,0.614685
2,0.532737,0.591901,6.977868e-05,0.7,0.226099
3,0.891039,0.893623,1.000000e+00,1.0,0.771676
4,0.121942,0.090771,0.000000e+00,0.1,0.181811


In [402]:
x_train = np.concatenate(( lg_oof_train,rf_oof_train, gb_oof_train, svc_oof_train, nn_oof_train), axis=1)
x_test = np.concatenate(( lg_oof_test,rf_oof_test, gb_oof_test, svc_oof_test, nn_oof_test), axis=1)
print('x_train.shape : ', x_train.shape)
print('x_test.shape : ', x_test.shape)

x_train.shape :  (700, 5)
x_test.shape :  (191, 5)


In [403]:
data_train_target.shape

(700,)

In [404]:
gb_st2 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1).fit(x_train, data_train_target)
predictions = gb_st2.predict(x_test)

In [405]:
sum(predictions == data_test_target)/len(predictions)

0.83246073298429324

In [406]:
predictions

array([ 1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1

In [407]:
data_test_target

array([ 1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,
        1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,
        1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1

In [414]:
test = []
for i,aaa in enumerate(nn_oof_test):
    if nn_oof_test[i] > 0.5 :
        test.append(1)
    else:
        test.append(0)

In [415]:
test

[1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0]

In [416]:
sum(test == data_test_target)/len(test)

0.84816753926701571

In [417]:
data_test_target

array([ 1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,
        1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,
        1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1